In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,9614
2,Huelva,Confirmados PDIA 14 días,1910
3,Huelva,Tasa PDIA 14 días,"373,9649882621984"
4,Huelva,Confirmados PDIA 7 días,749
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,16
629,Municipio de Huelva sin especificar,Total Confirmados,124
630,Municipio de Huelva sin especificar,Curados,84


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  9614.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2710.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 633 personas en los últimos 7 días 

Un positivo PCR cada 238 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,9614.0,749.0,1910.0,510743.0,146.649097,373.964988,264.0
Huelva-Costa,5304.0,508.0,1234.0,288115.0,176.318484,428.301199,183.0
Huelva (capital),2710.0,227.0,604.0,143663.0,158.008673,420.428364,91.0
Condado-Campiña,2961.0,195.0,540.0,155057.0,125.760204,348.259027,69.0
Isla Cristina,296.0,44.0,78.0,21264.0,206.922498,366.817156,25.0
Lepe,845.0,82.0,211.0,27431.0,298.931865,769.202727,23.0
Palos de la Frontera,199.0,32.0,59.0,11289.0,283.461777,522.632651,17.0
Cartaya,306.0,34.0,75.0,19974.0,170.221288,375.488135,16.0
Ayamonte,309.0,60.0,119.0,20946.0,286.450874,568.127566,14.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Corteconcepción,18.0,1.0,12.0,548.0,182.481752,2189.781022,1.0
Lucena del Puerto,70.0,15.0,30.0,3371.0,444.971818,889.943637,6.0
Niebla,112.0,18.0,33.0,4117.0,437.211562,801.554530,4.0
Lepe,845.0,82.0,211.0,27431.0,298.931865,769.202727,23.0
Palma del Condado (La),434.0,35.0,80.0,10761.0,325.248583,743.425332,12.0
Ayamonte,309.0,60.0,119.0,20946.0,286.450874,568.127566,14.0
Rociana del Condado,106.0,12.0,43.0,7855.0,152.768937,547.422024,3.0
Palos de la Frontera,199.0,32.0,59.0,11289.0,283.461777,522.632651,17.0
Paymogo,12.0,2.0,6.0,1159.0,172.562554,517.687662,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Almonte,427.0,4.0,65.0,24191.0,16.535075,268.694969,3.0,0.061538
Minas de Riotinto,62.0,1.0,15.0,3848.0,25.987526,389.812890,0.0,0.066667
Corteconcepción,18.0,1.0,12.0,548.0,182.481752,2189.781022,1.0,0.083333
Beas,51.0,2.0,17.0,4257.0,46.981442,399.342260,0.0,0.117647
Bonares,40.0,2.0,13.0,6058.0,33.014196,214.592275,1.0,0.153846
Puebla de Guzmán,21.0,1.0,6.0,3073.0,32.541490,195.248942,0.0,0.166667
Alosno,41.0,1.0,6.0,3923.0,25.490696,152.944175,0.0,0.166667
Bollullos Par del Condado,229.0,5.0,25.0,14272.0,35.033632,175.168161,1.0,0.200000
Trigueros,127.0,3.0,14.0,7713.0,38.895371,181.511733,0.0,0.214286
